# IMDB 영화평 감상분석

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(
    '../00.data/kaggle.com/word2vec-nlp-tutorial/labeledTrainData.tsv',
    header=0,
    sep='\t',
    quoting=3
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [3]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [4]:
import re

# remove br tag
df['review'] = df.review.str.replace('<br />', ' ')

# 영어 이외의 문자는 공백으로 변환
df['review'] = df.review.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [10]:
from sklearn.model_selection import train_test_split
feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size=0.3, random_state=156
)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## CountVectorizer

In [12]:
cv = CountVectorizer(
    stop_words='english',
    ngram_range=(1,2)
)
cv.fit(X_train.review)
X_train_vec = cv.transform(X_train.review)
X_test_vec = cv.transform(X_test.review)

In [14]:
lr = LogisticRegression(C=10)
lr.fit(X_train_vec, y_train)
pred = lr.predict(X_test_vec)
accuracy_score(y_test, pred)

0.886

## TfidfVectorizer

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1,2)
)
tv.fit(X_train.review)
X_train_vec = tv.transform(X_train.review)
X_test_vec = tv.transform(X_test.review)

In [18]:
lr = LogisticRegression(C=10)
lr.fit(X_train_vec, y_train)
pred = lr.predict(X_test_vec)
accuracy_score(y_test, pred)

0.8936

## 모델 저장하고 불러오기
<https://cloud.google.com/ai-platform/prediction/docs/exporting-for-prediction?hl=ko>

In [22]:
import joblib
joblib.dump(tv, './models/imdb_vec.pkl')
joblib.dump(lr, './models/imdb_lr.pkl')

['./models/imdb_lr']

In [23]:
# delete variables
del tv
del lr

In [24]:
new_vc = joblib.load('./models/imdb_vec.pkl')
new_lr = joblib.load('./models/imdb_lr.pkl')

In [25]:
new_X_train = new_vc.transform(X_train.review)
new_X_test = new_vc.transform(X_test.review)

In [26]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8936

## Pipeline을 써서 학습/예측/평가

In [27]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('cv', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr', LogisticRegression(C=10))
])
pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
score = accuracy_score(y_test, pred)
print('Count Vectorizer + Logistic Regression Accuracy: {}'.format(score))

Count Vectorizer + Logistic Regression Accuracy: 0.886


In [29]:
import joblib
joblib.dump(pipeline, './models/pipeline.pkl')

['./models/pipeline.pkl']

In [30]:
new_pipe = joblib.load('./models/pipeline.pkl')

In [31]:
pred = new_pipe.predict(X_test.review)
score = accuracy_score(y_test, pred)
print('Count Vectorizer + Logistic Regression Accuracy: {}'.format(score))

Count Vectorizer + Logistic Regression Accuracy: 0.886
